### Cluster Operator Generation Using Sympy

Sympy has a module called secondquant [documentation here](https://docs.sympy.org/latest/modules/physics/secondquant.html) and I want to use this to automatically generate some cluster operators. 
This was prompted by the repo [paggerq](https://github.com/edeprince3/pdaggerq) which I was told about by [Josh Goings](https://github.com/jjgoings) (whose blog and github site you should certainly look at). Let's start with
+ **Symbols**

    Quantum chemistry is pretty heavy on labels, we have lots of tensors, so how using sympy do we define some labels for our tensors? Unlike in Python where the first use of the variable declares it, in sympy we must declare our variables. So in Sympy we might write
```python
from sympy import *
i, j = symbols('i, j', cls=Dummy)
a, b, c = symbols('a:c', cls=Dummy)
```
But our labels usually mean something eg i,j will usually be occupied orbitals and a,b will be virtual orbitals. In the particle-hole formalism (PHF) \[*Shavitt & Bartlett 3.4*\] we say occupied levels (particles) are below the Fermi surface and virtual ones (holes) above. We can impose these conditions on our labels with
```python
i, j = symbols('i, j', below_fermi=True)
a, b = symbols('a, b', above_fermi=True)
```

+ **Creation and Annihilation Operators**

    An annihilation operator (usually denoted *a*) lowers by one the number of particles in a state. A creation operator (usually denoted *a*$^\dagger$) increases by one the number of particles in a state. They form an adjoint pair of operators. Second quantization uses these operators instead of wavefunctions. \[*Shavitt & Bartlett 3.6*\]. In sympy we define them by
```python
CreateFermion(p)*AnnihilateFermion(q)
or
Fd(p)*F(q)
```

+ **Normal Order**

    An important concept for creation and annihilation operators is normal order \[*Shavitt & Bartlett 3.89*\], the normal order of these operators is one in which all the creation operators come to the left of the annihilation operators. We can achieve this by progressively swapping adjacent operators making a change of sign of the overall expression each time. Sympy has an operator to do this
```python
NO(AnnihilateFermion(p)*CreateFermion(q))
```
For example

In [86]:
from sympy import *
from sympy.physics.secondquant import * 

p, q, r, s = symbols('p:s', cls=Dummy)

display(NO(Fd(s)*F(r)*F(q)*Fd(p)))

NO(CreateFermion(_p)*CreateFermion(_s)*AnnihilateFermion(_q)*AnnihilateFermion(_r))

+ **Tensors**

    We can define tensors, for example, the tensors $g_{ij}^{ab}$ and $f_i^a$ in Sympy by

In [87]:
i, j = symbols('i, j', below_fermi=True)
a, b = symbols('a, b', above_fermi=True)

g = AntiSymmetricTensor('g', (a, b), (i, j))
f = AntiSymmetricTensor('f', (a, ), (i, ))
display(g,f)

AntiSymmetricTensor(g, (a, b), (i, j))

AntiSymmetricTensor(f, (a,), (i,))

Note in the definition of AntiSymmetricTensor the contravariant indices come first (a, b) and the covariant index last (i, j).
+ **Normal Ordered Hamiltonian**

    We can now write code to form the normal ordered Hamiltonian \[*Shavitt & Bartlett 3.185*\]. If $f_a^i$ is the Fock operator and $v_{ij}^{ab}$ are the 2-electron repulsion integrals then we write the Normal Ordered Hamiltonian as $H_N = {f^p_q} p{\dagger}q$ + $\frac{1}{4}v^{pq}_{rs}p{\dagger}q{\dagger}sr$

In [88]:
p, q, r, s = symbols('p:s', cls=Dummy)
i, j = symbols('i,j', below_fermi = True)
a, b = symbols('a,b', above_fermi= True)

#Fock tensor in normal order
f = AntiSymmetricTensor('f', (p, ), (q, ))
pq = NO(Fd(p)*F(q))

#2-electron repulsion tensor in normal order
g = AntiSymmetricTensor('v', (p, q), (r, s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r))

#form Hamiltonian (normal order)
h = f * pq + Rational(1, 4) * g * pqsr

display(h)

AntiSymmetricTensor(f, (_p,), (_q,))*NO(CreateFermion(_p)*AnnihilateFermion(_q)) - AntiSymmetricTensor(v, (_p, _q), (_r, _s))*NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s))/4

+ **Contractions**

    The contraction of operators A and B is defined as A\*B - n\[A\*B\] , where n\[ \] means the normal order \[*Shavitt & Bartlett 3.95*\] and is written $\overbrace{AB}$. In sympy it's written 

 ```contraction(Fd(i),F(j))```
    

+ **Wick's Theorem**

    The time-independent form of Wick’s theorem states: A product of a string of creation and annihilation operators is equal to their normal product plus the sum of all possible normal products with contractions. In sympy

 ```wicks(Fd(p)*F(q))```
    
    
+ **Configuration Interaction**

    As an example of an application of all this let's look at CIS. The CIS energy is in second quantization terms $E_{cis} = <\phi_0|{i^\dagger}a H {b^\dagger}j|\phi_0>$, we program this in Sympy

In [89]:
w = wicks(Fd(i)*F(a)*h*Fd(b)*F(j), simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
display(w)

-KroneckerDelta(a, b)*AntiSymmetricTensor(f, (j,), (i,)) + KroneckerDelta(i, j)*AntiSymmetricTensor(f, (a,), (b,)) - AntiSymmetricTensor(v, (a, j), (b, i))

just need to mention
+ **Simplifying expressions**

    There are some useful functions which rationalise expressions. *evaluate\_deltas* does just that obeying Einstein summation convention. *substitute\_dummies* this routine simplifys Add expressions containing terms which differ only due to dummy variables. *keep_only\_fully\_contracted*, only fully contracted terms are returned. We'll see how to use these later. 


+ **Cluster Operators**

    The cluster operators are defined as $T_1=\sum t^a_i \lbrace{a^\dagger}i\rbrace$, $T_2=\frac{1}{4} \sum t^{ab}_{ij} \lbrace{a^\dagger}i{b^\dagger}j\rbrace$ etc \[*Shavitt & Bartlett 9.26-9.28, 9.29*\]. In Sympy we can do this as

In [90]:
def clusterOperators(level):
    if 'S' in level:
        i = symbols('i', below_fermi=True, cls=Dummy)
        a = symbols('a', above_fermi=True, cls=Dummy)
        ts = AntiSymmetricTensor('t', (a,), (i,))
        ai = NO(Fd(a)*F(i))
        t1 = ts * ai

    if 'D' in level:
        i, j = symbols('i,j', below_fermi=True, cls=Dummy)
        a, b = symbols('a,b', above_fermi=True, cls=Dummy)
        td = AntiSymmetricTensor('t', (a, b), (i, j))
        aibj = NO(Fd(a)*F(i)*Fd(b)*F(j))
        t2 = Rational(1, 4)*td*aibj

    if 'T' in level:
        i, j, k = symbols('i:k', below_fermi=True, cls=Dummy)
        a, b, c = symbols('a:c', above_fermi=True, cls=Dummy)
        tt = AntiSymmetricTensor('t', (a, b, c), (i, j, k))
        aibjck = NO(Fd(a)*F(i)*Fd(b)*F(j)*Fd(c)*F(k))
        t3 = Rational(1, 36)*tt*aibjck
      
    if level == 'S':   return t1
    if level == 'D':   return t2
    if level == 'SD':  return t1 + t2
    if level == 'SDT': return t1 + t2 + t3
    
display(clusterOperators('SD'))


AntiSymmetricTensor(t, (_a,), (_i,))*NO(CreateFermion(_a)*AnnihilateFermion(_i)) - AntiSymmetricTensor(t, (_a, _b), (_i, _j))*NO(CreateFermion(_a)*CreateFermion(_b)*AnnihilateFermion(_i)*AnnihilateFermion(_j))/4

+ **Similarity-Transformed Hamiltonian**

    This is defined as $e^{-T}{H_N}e^{T}$, where $H_N$ is the normal-ordered Hamiltonian and T are cluster operators. This is non-Hermitian giving rise to left and right eigenvectors.
    
    
+ **Commutator**

    The commutator bracket is defined \[A,B\] = AB - BA and in sympy,

    ```Commutator```
    
    
+ **Permutations**

    Sympy has a permutation operator, P(a,b) = $f_{ab} - f_{ba}$ 

    ```PermutationOperator(i,j)```
    
    
+ **Baker-Campbell-Hausdorff**

    The  Baker-Campbell-Hausdorff expansion (BCH) gives an expansion of $e^{-B}Ae^B$ \[*Shavitt \& Bartlett 10.4*\] and applied to $H_N$ \[*Shavitt & Bartlett 10.5*\]. Sympy doen't has a built-in version of this so we have to do some work. Below is a subroutine to do a BCH, note that many of the simplification flags we used before can also be functions. The $expand$ method of a expression performs all eg the multiplication brackets and reduces the expression to simple terms.

In [91]:
def bakerCampbellHausdorff(h, level, degree):
    
    symbols = {'above': 'defg','below': 'lmno', 'general': 'pqrst' }

    #commutator bracket
    c = Commutator
   
    bch = zeros(5)
    bch[0] = h
    for i in range(degree):
        t  = clusterOperators(level)
        bch[i+1] = wicks(c(bch[i], t))
        bch[i+1] = evaluate_deltas(bch[i+1])          
        bch[i+1] = substitute_dummies(bch[i+1])

    #BCH series
    BCH = bch[0] + bch[1] + bch[2]/2 + bch[3]/6 + bch[4]/24 

    #tidy up and compact
    BCH = BCH.expand()
    BCH = evaluate_deltas(BCH)
    BCH = substitute_dummies(BCH, new_indices=True,
                                  pretty_indices=symbols)
    
    return BCH


+ **Coupled-Cluster Doubles**

    Let's see if we can use the above to get the CCD energy and amplitude expressions. We need to evaluate<br>
$E_{cc} = <\phi_0 | e^{-t_2}{H_N}e^{t_2} | \phi_0>$<br>
$T_2 = <\phi_0 | {i^\dagger}{j^\dagger}{ba} \ e^{-t_2}{H_N}e^{t_2} | \phi_0>$

In [105]:
#get the normal ordered hamiltonian from earlier

#do Baker-Campbell-Hausdorff on H
ccd = bakerCampbellHausdorff(h, 'D', 4)

#cluster energy
w = wicks(ccd, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
symbol_rules = {'below':'ijklmno', 'above': 'abcdef', 'general':'pqrstu'}
ccEnergy = substitute_dummies(w, new_indices=True, pretty_indices=symbol_rules)

#doubles amplitudes
expression = wicks(Fd(i)*Fd(j)*F(b)*F(a)*ccd, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
symbol_rules = {'below':'klmno', 'above': 'cdef', 'general':'pqrstu'}
td = substitute_dummies(expression, new_indices=True, pretty_indices=symbol_rules)
p = [PermutationOperator(i,j), PermutationOperator(a,b)]
doubles = simplify_index_permutations(td, p)

display(ccEnergy)
display(doubles)

AntiSymmetricTensor(t, (_a, _b), (_i, _j))*AntiSymmetricTensor(v, (_i, _j), (_a, _b))/4

AntiSymmetricTensor(f, (_k,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _k))*PermutationOperator(i, j) - AntiSymmetricTensor(f, (a,), (_c,))*AntiSymmetricTensor(t, (b, _c), (i, j))*PermutationOperator(a, b) + AntiSymmetricTensor(t, (_c, _d), (i, j))*AntiSymmetricTensor(t, (a, b), (_k, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))/4 + AntiSymmetricTensor(t, (_c, _d), (i, j))*AntiSymmetricTensor(v, (a, b), (_c, _d))/2 + AntiSymmetricTensor(t, (_c, _d), (j, _k))*AntiSymmetricTensor(t, (a, b), (i, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))*PermutationOperator(i, j)/2 + AntiSymmetricTensor(t, (a, _c), (i, _k))*AntiSymmetricTensor(t, (b, _d), (j, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))*PermutationOperator(i, j) + AntiSymmetricTensor(t, (a, _c), (i, _k))*AntiSymmetricTensor(v, (b, _k), (j, _c))*PermutationOperator(a, b)*PermutationOperator(i, j) - AntiSymmetricTensor(t, (a, _c), (i, j))*AntiSymmetricTensor(t, (b, _d), (_k, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))*Permutat

+ **Parsing The Output**

    The actual returned values (not processed by *display*) are comprised of terms like
       
    ```AntiSymmetricTensor(f, (_k,), (i,))*AntiSymmetricTensor(t, (a, b), (j,_k))*PermutationOperator(i, j) + AntiSymmetricTensor(f, (a,), (_c,))*AntiSymmetricTensor(t, (b, _c), (i, j))*PermutationOperator(a, b) + AntiSymmetricTensor(t, (_c, _d), (i, j))*AntiSymmetricTensor(t, (a, b), (_k, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))/4```

    The energy and amplitudes are each returned as a single string. The main tool we have is *args*, *(expr).args* returns a tuple of each term in the string. Each element in the tuple could be a term with multiple tensors (eg  AntiSymmetricTensor(t, (_c,_d), (i, j)) * AntiSymmetricTensor(v, (a, b), (_c, _d))/2) or a single tensor (eg + AntiSymmetricTensor(v, (a, b), (i, j))). (It could in some situations be a constant). You can tell which sort it is using *isinstance*. The types we can test for with *isinstance* are Add, Mul, AntiSymmetricTensor, Rational, KroneckerDelta and PermutationOperator.

    Let's see if we can parse doubles. First we decide if we have a single item or a multiple one, then if it's multiple we make a list out of it's components. The list will typically contain \[an optional multiplier,AntiSymmetricTensors,..., PermutationOperator\]. Here's an example from the doubles.

In [93]:
if isinstance(doubles, Mul) or isinstance(doubles, AntiSymmetricTensor): 
    items = [doubles]
else:
    items = doubles.args
    
units = []
for item in items:

    #multiple component term
    if isinstance(item, Mul):
        subUnits = []
        for atom in item.args:
            subUnits.append(atom)
        units.append(subUnits)

    #kronecker deltas
    elif isinstance(item, KroneckerDelta): 
        units.append([item]) 
    #single tensor term
    elif isinstance(item, AntiSymmetricTensor):
        units.append([item])
    else:
        print('No handler for class ', type(item),' ', item)
display(units[3])

[-1,
 AntiSymmetricTensor(f, (a,), (_c,)),
 AntiSymmetricTensor(t, (b, _c), (i, j)),
 PermutationOperator(a, b)]

Now we've split the expression into a list for each term, we need to interpret the individual list elements. We test each  list item against 
+ isinstance(\_ , Number)
+ isinstance(\_ , AntisymmetricTensor)
+ isinstance(\_ , PermutationOperator)
+ isinstance(\_ , KroneckerDelta)

If we have an AntisymmetricTensor we can find 
+ label as \_.symbol
+ contravariant indices as \_.upper
+ covariant indices as \_.lower

If we have a KroneckerDelta or PermutationOperator then
+ the indices are \_.args

For example,

In [94]:
for item in units[3]:
    if isinstance(item, Number):
        print(Float(item))
    if isinstance(item, AntiSymmetricTensor):
        print(item.symbol, item.upper, item.lower)
    if isinstance(item, PermutationOperator) or isinstance(item, KroneckerDelta):
        print(item.args, item.args[0], item.args[1])

-1.00000000000000
f (a,) (_c,)
t (b, _c) (i, j)
(a, b) a b


So we can parse the expressions into something like ```-1.0000 * np.einsum('ac,bcij->abij', f[v, o], t)```.\
These are the expressions that need to be evaluated for a variety of coupled-cluster methods

+ **CCD**
    + $E = <\phi_0|e^{-t_2}H_Ne^{t_2}|\phi_0>$
    + $T_2 = <\phi_0|{i^\dagger}{j^\dagger}ba \ e^{-t_2}H_Ne^{t_2}|\phi_0>$
    
+ **CCSD**
    + $E = <\phi_0|e^{-(t_1+t_2)}H_Ne^{(t_1+t_2)}|\phi_0>$
    + $T_1 = <\phi_0|{i^\dagger}b \ e^{-(t_1+t_2)}H_Ne^{(t_1+t_2)}|\phi_0>$
    + $T_2 = <\phi_0|{i^\dagger}{j^\dagger}ba \ e^{-(t_1+t_2)}H_Ne^{(t_1+t_2)}|\phi_0>$

+ **CCSDT**
    + $E = <\phi_0|e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_1 = <\phi_0|{i^\dagger}b \ e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_2 = <\phi_0|{i^\dagger}{j^\dagger}ba \ e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_3 = <\phi_0|{i^\dagger}{j^\dagger}{k^\dagger}cba \ e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$

+ **CCSD(T)**
    + $E = <\phi_0|e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_1 = <\phi_0|{i^\dagger}b \ e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_2 = <\phi_0|{i^\dagger}{j^\dagger}ba \ e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_3 = <\phi_0|{i^\dagger}{j^\dagger}{k^\dagger}cba \ (e^{-t_3}f_Ne^{t_3} + [v,t_2])|\phi_0>$

    
+ **CC2**
    + $E = <\phi_0|e^{-(t_1+t_2)}H_Ne^{(t_1+t_2)}|\phi_0>$
    + $T_1 = <\phi_0|{i^\dagger}b \ e^{-(t_1+t_2)}H_Ne^{(t_1+t_2)}|\phi_0>$
    + $T_2 = <\phi_0|{i^\dagger}{j^\dagger}ba \ (e^{-t_2}fe^{t_2} + e^{-t_1}ve^{t_1})|\phi_0>$
    
+ **CC3**
    + $E = <\phi_0|e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_1 = <\phi_0|{i^\dagger}b \ e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_2 = <\phi_0|{i^\dagger}{j^\dagger}ba \ e^{-(t_1+t_2+t_3)}H_Ne^{(t_1+t_2+t_3)}|\phi_0>$
    + $T_3 = <\phi_0|{i^\dagger}{j^\dagger}ba \ (e^{-(t_1+t_2+t_3)}fe^{(t_1+t_2+t_3)} + e^{-t_1}ve^{t_1} + v + [v,t_2] + [[v,t_1],t_2] + \frac{1}{2}[[[v,t_1],t_1],t_2] + \frac{1}{6}[[[[v,t_1],t_1],t_1],t_2])|\phi_0>$
    
    
+ **LCCD**
    + Use CCD but restrict the Baker-Campbell-Hausdorff expansion to linear terms ie ```bakerCampbellHausdorff(h,'D',1)```
    
+ **LCCSD**
    + Use CCSD but restrict the Baker-Campbell-Hausdorff expansion to linear terms ie ```bakerCampbellHausdorff(h,'SD',1)```
    


+ **CCSD-&Lambda;**

    The CCSD Lagrangian is given by $\mathfrak{L} = <\phi_0| (1+\Lambda) \ e^{-(t_1+t_2)}H_Ne^{(t_1+t_2)} |0>$

    The &Lambda; are de-excitation operators defined as $L_1=\sum L^i_a \lbrace{i\dagger}a\rbrace$, $T_2=\frac{1}{4} \sum l^{ij}_{ab} \lbrace{a^\dagger}i{b^\dagger}j\rbrace$ \[*Shavitt & Bartlett 13.14*] etc 

    The derivative of the Lagrangian with respect to $T_1  (t_1 \lbrace{a^\dagger}i\rbrace)$ is $<0| {-a^\dagger} i  e^{-T}{H_N}e^T |0> + <0| e^{-T}{H_N}e^T \lbrace{a^\dagger}i\rbrace|0> + <0|\mathfrak{L}  -e^{-T}\lbrace{a^\dagger}i\rbrace  {H_N}e^T |0> + <0| \mathfrak{L} e^{-T}{H_N}\lbrace{a^\dagger}i\rbrace  e^T |0>$\
which is $<0| e^{-T}{H_N}e^T |\psi^a_i> + <0| \mathfrak{L} e^{-T} [H_N,\lbrace{a^\dagger}i\rbrace] e^T |0> $<br><br>
The derivative of the Lagrangian with respect to $T_2  (t_2 \lbrace{a^\dagger}{b^\dagger}ij\rbrace)$ is\
$<0| (-\lbrace {a^\dagger}{b^\dagger}ij\rbrace  e^{-T}H_Ne^T |0> + <0| e^{-T}H_Ne^T  \lbrace {a^\dagger}{b^\dagger}ij\rbrace|0> + <0|\mathfrak{L} (-e^{-T}\lbrace {a\dagger}{b\dagger}ij\rbrace  H_Ne^T |0> + <0| \mathfrak{L} e^{-T}H_N\lbrace {a^\dagger}{b^\dagger}ij\rbrace |0>$
which is $<0| e^{-T}H_Ne^T |\phi^{ab}_{ij}> + <0| \mathfrak{L} e^{-T} [H_N,\lbrace{a^\dagger}{b^\dagger}ij\rbrace] e^T |0>$ 

    This is implemented along the lines of...
```
st = bakerCampbellHausdorff(h*Fd(a)*F(i),'SD',4)
w = wicks(st , simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
st = bakerCampbellHausdorff(Commutator(h, Fd(a)*F(i)),'SD',4)
leftOperators = lagrangeOperator('S') + lagrangeOperator('D')
w += wicks(leftOperators*st, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
```
    and doubles...
```python
st = bakerCampbellHausdorff(h*Fd(a)*Fd(b)*F(j)*F(i),'SD',4)
w = wicks(st, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
st = bakerCampbellHausdorff(Commutator(h,Fd(a)*Fd(b)*F(j)*F(i)),'SD',4)
leftOperators = lagrangeOperator('S') + lagrangeOperator('D')
w += wicks(leftOperators*st, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
```
    The Lagrange amplitudes are given by eg...
```python
if 'D' in level:
    #Lagrange doubles amplitudes
    i, j = symbols('i,j', below_fermi=True, cls=Dummy)
    a, b = symbols('a,b', above_fermi=True, cls=Dummy)
    l2 = Rational(1, 4) * AntiSymmetricTensor('l2', (i, j), (a, b)) *  NO(Fd(i)*F(a)*Fd(j)*F(b))
```

We gave the Lagrangian earlier as $\mathfrak{L} = <0| (1+\Lambda) \ e^{-(t_1+t_2)}H_Ne^{(t_1+t_2)} |0>$, hence the Lagrangian energy is given by
$<0| e^{-T}H_Ne^T |0> + <0| L_1 e^{-T}H_Ne^T |0> + <0| L_2 e^{-T}H_Ne^T  |0>$, or\
$E_{ccsd} + l1<0|\lbrace{i^\dagger}a\rbrace e^{-T}H_Ne^T  |0> + l2<0| \lbrace{i^\dagger}{j^\dagger}ba\rbrace e^{-T}H_Ne^T  |0>$ or\
$E_{ccsd} + l1 CCSD_{singles} + l2 CCSD_{doubles}$


+ **CCSD Response Density Matrices**

    \[*Shavitt & Bartlett 11.88*\] defines the one-particle response density matrix (oprdm) as $\gamma_{qp} = <0| (1 + \Lambda)e^{-T}\lbrace{p^\dagger}q\rbrace \ e^T |0>$ (the 1 is $\Lambda_0$)

We can generalise the density matrix equation to include EOM cases

| method | $\Lambda_0$  | $\Lambda_1$  | $\Lambda_2$  |
|:---:|:--:|:-----------:|:------:|
| CC | $1$ | $\lambda^i_a \lbrace{i^\dagger}a\rbrace$     | $\frac{1}{4}\lambda^{jk}_{bc} \lbrace{j^\dagger}{k^\dagger}cb\rbrace$  |
| EE | $0$ | $\lambda^i_a \lbrace{i^\dagger}a\rbrace$     | $\frac{1}{4}\lambda^{jk}_{bc} \lbrace{j^\dagger}{k^\dagger}cb\rbrace$  |
| IP | $0$ | $\lambda^i \lbrace{i^\dagger}\rbrace$     | $\frac{1}{4}\lambda^{jk}_{c} \lbrace{j^\dagger}{k^\dagger}c\rbrace$  |
| EA | $0$ | $\lambda_a \lbrace a\rbrace$     | $\frac{1}{4}\lambda^{k}_{bc} \lbrace{k^\dagger}cb\rbrace$  |

The basic EE form has $\lbrace {i^\dagger}a\rbrace$. if we go to IP we lose the hole so go to ${i^\dagger}$ or if we go to electron affinity we lose the particle $a$. There are also double IP (DIP) and double EA (DEA).\
In EOM the density matrices are given by $<0| L_k e^{-T} \lbrace{p^\dagger}q\rbrace e^T R_k |0>$ \[*Shavitt and Bartlett 13.28*]. Here L and R are the left and right eigenvectors \[*Shavitt & Bartlett 13.14 and 13.9*], they represent de-excitations and excitations, respectively. They form of bi-orthonormal set \[*Shavitt & Bartlett 13.16*]. For the response density we only need the de-excitation operators which we can program as
```python
def de_excitations(method):

   i, j, k = symbols('i:k' ,below_fermi=True, cls=Dummy)
   a ,b, c = symbols('a:c' ,above_fermi=True, cls=Dummy)   

   if method == 'IP':
       return [0, AntiSymmetricTensor('l',(i,),())*Fd(i), Rational(1, 2)* \
                  AntiSymmetricTensor('l',(j,k),(a,))*Fd(j)*Fd(k)*F(a)]  
   elif method == 'EA':
       return [0, AntiSymmetricTensor('l',(),(a,))*F(a), Rational(1, 2)* \
                  AntiSymmetricTensor('l',(i,),(b,c))*Fd(i)*F(c)*F(b)]
   elif method == 'EE':
       return [0, AntiSymmetricTensor('l',(i,),(a,))*Fd(i)*F(a), Rational(1, 4)* \
                  AntiSymmetricTensor('l',(j,k),(b,c))*Fd(j)*Fd(k)*F(c)*F(b)]
   elif method == 'CC':
       return [1, AntiSymmetricTensor('l',(i,),(a,))*Fd(i)*F(a), Rational(1, 4)* \
                  AntiSymmetricTensor('l',(j,k),(b,c))*Fd(j)*Fd(k)*F(c)*F(b)]
```
Then if we wanted the occupied-occupied block $\gamma_{ij}$
```python
L = sum(de_excitations(method))

cc = bakerCampbellHausdoff(Fd(i)*F(j),'SD',4)
evaluate_deltas(wicks(L*cc, keep_only_fully_contracted = True))
cc = substitute_dummies(cc, new_indices=True, pretty_indices = {'below':  'klmno','above':  'abcde'})
```

The 2-particle response density matrix (tprdm) is given by $\Gamma_{pqrs} = <0|(1+\Lambda)e^{-T} \lbrace{p^\dagger}{q^\dagger}sr\rbrace e^T|0>$ \[*Shavitt & Bartlett 11.89*\] and implemented as, for example, $\Gamma_{ijkl}$

```python
i,j,k,l = symbols('i:l' , below_fermi=True)
a,b,c,d = symbols('a:d' , above_fermi=True)
p = [PermutationOperator(i,j), PermutationOperator(a,b)]

cc = bakerCampbellHausdorff(Fd(i)*Fd(j)*F(l)*F(k),'SD',4)
oooo = wicks(L*cc , keep_only_fully_contracted=True, simplify_kronecker_deltas = True)
oooo = simplify_index_permutations(oooo, [PermutationOperator(i,j), PermutationOperator(k,l)])
oooo = substitute_dummies(oooo,new_indices=True, pretty_indices={'below':  'mnop','above':  'abcde'})
```

+ **EOM**
    The Equation-of-Motion solution requires, in addition to the left-hand de-excitation operators we defined above, a set of right-hand excitation operators - $R_k$ defined to act as $R_k |\phi_0> = |\phi_k>$ \[*Shavitt & Bartlett 13.7*\].
    These are can be defined as
```python
def excitationOperators(level):

    i, j, k = symbols('i,j,k' ,below_fermi=True, cls=Dummy)
    a ,b, c = symbols('a:c' ,above_fermi=True, cls=Dummy)   

    if level == 'IP':
        return [0, AntiSymmetricTensor('r',(),(i,))*F(i), Rational(1, 2)*AntiSymmetricTensor('r',(a,),(j,k))*Fd(a)*F(k)*F(j)]
    elif level == 'EA':
        return [0, AntiSymmetricTensor('r',(a,),())*Fd(a), Rational(1, 2)*AntiSymmetricTensor('r',(b,c),(i,))*Fd(b)*Fd(c)*F(i)]
    elif level == 'EE':
        return [AntiSymmetricTensor('r0',(),()), AntiSymmetricTensor('r',(a,),(i,))*Fd(a)*F(i), \
                Rational(1, 4)*AntiSymmetricTensor('r',(b,c),(j,k))*Fd(b)*Fd(c)*F(k)*F(j) ]
    elif level == 'CC':
        return [1, 0, 0]
```
Our main equation is $[H, R_k]|0> = \omega_k R_k|0>$ \[*Shavitt & Bartlett 13.20*\].

To compute the singles-singles block of EE-EOM-CC then we would do
```python
R = excitationOperators('EE')
qOperators, qSymbols = [Fd(i)*F(a), {'below': 'jklmno','above': 'bcdefg'}]

ss = evaluate_deltas(wicks(qOperators*Commutator(h,R[1]) , keep_only_fully_contracted=True))
ss = substitute_dummies(ss, new_indices=True, pretty_indices= qSymbols)

p = [PermutationOperator(i,j), PermutationOperator(a,b)]
block['ss'] = simplify_index_permutations(ss, p)
```
For \[ds\] and \[dd\] blocks R\[1\] becomes R\[2\] and for IP and EA for \[ss\] and \[sd\] use<br>
if level == 'IP': qOperators, qSymbols = \[Fd(i), {'below': 'jklmno','above': 'abcdefg'}\]<br>
if level == 'EA': qOperators, qSymbols = \[F(a), {'below': 'ijklmno','above': 'bcdefg'}\]

and for \[ds\] and \[dd\] blocks\
if level == 'IP': qOperators, qSymbols = \[Fd(i)\*Fd(j)\*F(a), {'below': 'klmno','above': 'bcdefg'}\]<br>
if level == 'EA': qOperators, qSymbols = \[Fd(i)\*F(b)\*F(a), {'below': 'jklmno','above': 'cdefg'}\]<br>
if level == 'EE': qOperators, qSymbols = \[Fd(i)\*Fd(j)\*F(b)\*F(a), {'below': 'klmno','above': 'cdefg'}\]

As an example below is EE-EOM-CCSD for the singles-doubles block..

In [95]:
i, j, k = symbols('i, j, k', below_fermi = True)
a, b, c = symbols('a, b, c', above_fermi= True)
qOperators, qSymbols = [Fd(i)*F(a), {'below': 'jklmno','above': 'bcdefg'}]

st = bakerCampbellHausdorff(h,'SD',4)
R2 = Rational(1, 4)*AntiSymmetricTensor('r',(b,c),(j,k))*Fd(b)*Fd(c)*F(k)*F(j) 
sd = evaluate_deltas(wicks(qOperators*Commutator(st,R2) , keep_only_fully_contracted=True))
sd = substitute_dummies(sd, new_indices=True, pretty_indices= qSymbols)
p = [PermutationOperator(i,j), PermutationOperator(a,b)]

display(simplify_index_permutations(sd, p))


AntiSymmetricTensor(f, (_j,), (_b,))*AntiSymmetricTensor(r, (a, _b), (i, _j)) + AntiSymmetricTensor(r, (_b, _c), (i, _j))*AntiSymmetricTensor(t, (a,), (_k,))*AntiSymmetricTensor(v, (_j, _k), (_b, _c))/2 + AntiSymmetricTensor(r, (_b, _c), (i, _j))*AntiSymmetricTensor(v, (a, _j), (_b, _c))/2 + AntiSymmetricTensor(r, (a, _b), (_j, _k))*AntiSymmetricTensor(t, (_c,), (i,))*AntiSymmetricTensor(v, (_j, _k), (_b, _c))/2 - AntiSymmetricTensor(r, (a, _b), (_j, _k))*AntiSymmetricTensor(v, (_j, _k), (i, _b))/2 + AntiSymmetricTensor(r, (a, _b), (i, _j))*AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(v, (_j, _k), (_b, _c))

This is taken from working out the EOM equations from original papers
#### H<sub>SD</sub>
$[F_{ld} \delta_{ik} \delta_{ac} + 0.5 W_{alcd} \delta_{ik} - 0.5 W_{klid} \delta_{ac}] r^{lkcd}$

+ $+F_{ld} = f_{ld} \delta_{ik}\delta_{ac} + t^e_m g_{lmde} \delta_{ik} \delta_{ac}$

    + [16] $+f_{ld} \delta_{ik} \delta_{ac} ... ^{16}$ 
    + [17] $+t^e_m g_{lmde} \delta_{ik} \delta_{ac} ...^{21}$
    
+  $+0.5 W_{alcd} = 0.5 g_{alcd}  \delta_{ik} - 0.5 t^a_m g_{mlcd} \delta_{ik}$

    + [18] $+0.5 g_{alcd} \delta_{ik} ...^{17}$
    + [19] $-0.5 t^a_m g_{mlcd} \delta_{ik} ...^{20}$

+ $-0.5 W_{klid} = -0.5 (g_{klid} \delta_{ad} + t^e_i g_{kled} \delta_{ac})$

    + [20] $-0.5 g_{klid} \delta_{ac} ...^{18}$
    + [21] $-0.5 t^e_i g_{kled} \delta_{ac} ...^{19}$

We can see these equations are the same as the Sympy derived ones eg $r_{ij}^{ab} t^c_k v_{bc}^{jk}$ if we write out the einsum string 'ijab,ck,bcjk->ai' , we see c and k are repeated indices so we may write (k->m, c->e) 'ijab,em,bejm->ai', b and j are also repeated so (b->d, j->l), 'ilad,em,delm->ai', using symmetries of 2-electron integrals, 'adil,em,lmde->ai' or 'cdkl,em,lmde,ac,ik->ai' where ik and ac are $\delta$. This is equation \[17\]. 


+ **EOM-MBPT2**

It is easy to get EOM-MBPT2 level results from, for example, EE-EOM. Set $t_1 = 0$, $t_2 = <ab||ij>* (\epsilon[i,i]+\epsilon[j,j]-\epsilon[a,a]-\epsilon[b,b])^{-1}$ and $f^a_i=0$.

+ **LR-CC2**

This is defined the same as EOM-CCSD for singles-singles and singles-doubles blocks. The doubles-singles block is defined as $<\phi^{ab}_{ij}| e^{t_1}H_N e^{t_1} | \phi^c_k>$ and the doubles-doubles block as $<\phi^{ab}_{ij}| e^{t_2}F_N e^{t_2} | \phi^{cd}_{kl}>$. The only differences here are to use ```bakerCampbellHausdorff(h,'S',4)``` instead of 'SD' for doubles-singles and to use ```bakerCampbellHausdorff(f,'D',4)``` for doubles-doubles.

That's all for now.